### GLM Classification

This notebook shows how to do classification with Julia's GLM package.  The data comes from Chap 4 of the ISL book. 

In [1]:
import Pkg; using Pkg; 
#Pkg.add("Makie");
#Pkg.add("GLMakie")

#Pkg.add("CairoMakie")
#Pkg.update()



In [1]:

using CSV
using DataFrames
using Plots
using StatsPlots
using GLM
using Statistics
using Distributions
using Random
using MultivariateStats
using Makie, GLMakie
# using GLMakie

In [2]:

#=
		function readCSVFile( fileName::String, addOnes="NO" )::DataFrame

	Read a CSV file located at string given as argument.
	If addOnes is omitted the default is "NO"	
	If addOnes == "NO"  return a DataFrame obj that contains the data in the file.
	If addOnes == "YES" return a DataFrame obj that contains ones in first column and
	the data from the CSV file.
=#
function readCSVFile( fileName::String, addOnes="NO" )::DataFrame

	if addOnes== "NO"
		df = DataFrame(CSV.File( fileName))
		return df
	end

	if addOnes== "YES"
		df1 = DataFrame(CSV.File( fileName))

		nr = size(df1,1)
		#create a df of size (nr, 1) with ones in first column
		df2 = DataFrame(ones(nr, 1), :auto)
		# return a df with ones and the content from the CSV file
		df = hcat(df2, df1)
		#rename first column as "x0" to be consistent with LinReg
		n = names(df)
		rename!(df, n[1] => "x0")
		return df
	end
end

readCSVFile (generic function with 2 methods)

In [3]:

#=
			function dfConfig( df::DataFrame, cv::Vector )::DataFrame
		Receive as arguments a df and a vector. The columns from the df 
		identified in the vector are included in the new df object returned.
		Example:

			v = [2,3,4,5,6]
			dfNew = dfConfig(df, v)
			# dfNew will have the columns identified in v
=#
function dfConfig(df::DataFrame, cv::Vector) :: DataFrame
	return select(df, cv)
end

dfConfig (generic function with 1 method)

In [9]:
#=
          function defaultFig41()
 The ISLR V2 Default credit data set contains 10,000 rows with the following
  columns:   
            ColNumber default	student	balance	income
 
  The default column contains "YES" / "NO" values that need to be translated into 0.0 / 1.0 

  Using Makie
  
  Take a look at https://docs.makie.org/stable/tutorials/getting-started#Getting-started

=#

using Makie; using CairoMakie

function defaultFig41()


    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Default.csv", "NO")
    nr = size(df, 1)
    nc = size(df, 2)
    print("df size =", nr, "  nc =", nc)
  
    # The data columns are [RecordNumber, Default[Y,N], 	Student[Y,N] Balance, 	Income]
    dfc   = dfConfig( df, [1] )   # get column 1, which contains row number
    dfd   = dfConfig( df, [2] )   # get Default
    dfs   = dfConfig( df, [3] )   # get Student
    dfb   = dfConfig( df, [4] )   # get Balance
    dfi   = dfConfig( df, [5] )   # get Income 
  
    ### Replace the elements in Column 1 so that they contain 1.0 as number
    nr = size(dfc,1)
    dfc = DataFrame(ones(nr, 1), :auto)
  
    ### Replace the elements of type string that contain "Yes" or "No" with a number
    ### so that logit regression could be computed on this data.
    
    #replace!(dfd.Default, "Yes" => "1")
    #replace!(dfd.Default, "No"  => "0")
    #dfd[!, :Default] = parse.(Int,dfd[!, :Default])
  
    xDf = hcat( dfs, dfb, dfi)

    X = Matrix(xDf) 
    Y = Matrix( dfd )

    #f = Figure()
    #ax = Makie.Axis(f[1, 1], xlabel = "x label", ylabel = "y label",
    #title = "Title")
    #Makie.scatter!(0:0.5:10, cos, color = :orange)
    #f

    f1 = Figure()
    ax = Makie.Axis(f1[1, 1], title="Default Data: Income vs Balance" , xlabel="Balance", ylabel="Income")
    Makie.scatter!(marker=:circle, markersize=10, color=Y[1:10000],   X[1:10000,2], X[1:10000,3] ) # Balance vs Income 
    f1 
    Makie.save("/hm2/code/julia/LearnGLM/figs/ISL_Fig41a.png", f1)
   

    f3 = Figure()
    ax = Makie.Axis(f3[1, 1], title="Default Data: Default | Income" , xlabel="Income", ylabel="Default")
    Makie.scatter!(ax, marker=:circle, markersize=3, color=Y[1:10000],   X[1:10000,3], Y[1:10000] ) #Default | Income 
    # f3
    save("/hm2/code/julia/LearnGLM/figs/ISL_Fig41c.png", f3)

    f4 = Figure()
    ax = Makie.Axis(f4[1, 1], title="Default Data: Default | Balance" , xlabel="Balance", ylabel="Default")
    Makie.scatter!(ax, marker=:circle, markersize=3, color=Y[1:10000],   X[1:10000,2], Y[1:10000] ) #Default | Income 
    # f4
    save("/hm2/code/julia/LearnGLM/figs/ISL_Fig41d.png", f4)

    f5 = Figure()
    ax = Makie.Axis(f5[1, 1], title="Default Data: Balance vs Default" , xlabel="Balance", ylabel="Default")
    Makie.scatter!(ax, marker=:circle, markersize=3, color=Y[1:10000],   X[1:10000,2], Y[1:10000] ) # Balance vs Default 
    # 
    save("/hm2/code/julia/LearnGLM/figs/ISL_Fig42a.png", f5)
end


defaultFig41()

df size =10000  nc =5

CairoMakie.Screen{IMAGE}


In [ ]:

function defaultDataTable4123()

    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Default.csv", "NO")
    nr = size(df, 1)
    nc = size(df, 2)
    print("df size =", nr, "  nc =", nc, "\n")
  
    dfc   = dfConfig( df, [1] )   # get column 1, which contains row number
    dfd   = dfConfig( df, [2] )   # get Default
    dfs   = dfConfig( df, [3] )   # get Student
    dfb   = dfConfig( df, [4] )   # get Balance
    dfi   = dfConfig( df, [5] )   # get Income 
  
    ### Replace the elements in Column 1 so that they contain 1.0 as number
    nr = size(dfc,1)
    dfc = DataFrame(ones(nr, 1), :auto)
  
    ### Replace the elements of type string that contain "Yes" or "No" with a number
    ### so that logit regression could be computed on this data.
    
    #replace!(dfd.Default, "Yes" => "1")
    #replace!(dfd.Default, "No"  => "0")
    #dfd[!, :Default] = parse.(Int,dfd[!, :Default])
  
    dfy = hcat( dfd, dfs, dfb, dfi)
  
    ### The formula below produces same results as Table 4.1, pp. 136 from the ISLR2 book.
    ### Note that ProbitLink() is NOT used in the "R" code of the book
    fm = @formula(Default ~ Balance)
    logit = glm(fm, dfy, Binomial())
    println(" Table 4.1")
    println(logit)
  
    ### The formula below produces same results as Table 4.2, pp. 137 from the ISLR2 book.
    ### Note1: ProbitLink() is NOT used in the "R" code of the book
    ### Note2: The "student" column contains values "Yes" / "No". 
    ###        This is OK when the column is part of the X parameters
    ### Note3: The "default" column contains values "Yes" / "No"
    ###        Thuis is NOT OK when the column is the independent variable 
    println("\n\n Table 4.2")
    fm = @formula(Default ~ Student)
    logit = glm(fm, dfy, Binomial())
    println(logit)
 
    # I added this formula to get the betas dor Default | income
    println("\n\n Table 4.2x ")
    fm = @formula(Default ~ Income)
    logit = glm(fm, dfy, Binomial())
    println(logit)
 
    ### The formula below produces same results as Table 4.3, pp. 138 from the ISLR2 book.
    println("\n\n Table 4.3")
    fm = @formula(Default ~ Balance + Income + Student)
    logit = glm(fm, dfy, Binomial())
    println(logit)

  end

defaultDataTable4123()  
println(" Execute cell" )

df size =10000  nc =5
 Table 4.1
StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Default ~ 1 + Balance

Coefficients:
────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error       z  Pr(>|z|)   Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)  -10.6513      0.361157    -29.49    <1e-99  -11.3592    -9.94348
Balance        0.00549892  0.00022037   24.95    <1e-99    0.005067   0.00593083
────────────────────────────────────────────────────────────────────────────────


 Table 4.2
StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, 

In [7]:
# p(default | balance) 
function defaultDataFig42b(b0, b1)

    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Default.csv", "NO")

    dfd   = dfConfig( df, [2] )   # get Default
    dfb   = dfConfig( df, [4] )   # get Balance

    nr = nrow(dfb)

    X = Matrix( dfb ) 
    Y = Matrix( dfd )
    ylr = zeros(Float64, nr)

    for i = 1:nr 
        num = exp(b0 + b1*X[i])
        ylr[i] = num / (1.0 + num)
     end

    f = Figure()
    ax = Makie.Axis(f[1, 1], title="Log Reg       Default | Balance" , xlabel="Balance", ylabel="Default")
    Makie.scatter!(ax, marker=:circle, markersize=3, color=:blue,  X[1:10000], ylr[1:10000] ) # Default | Balance 
    # lines!(ax, X[1:10000], ylr[1:10000])
    
    ax = Makie.Axis(f[1, 1])
    Makie.scatter!(ax, marker=:circle, markersize=3, color=:red,  X[1:10000], Y[1:10000] ) #  Default | Balance

    hideydecorations!(ax, ticks = false)  # Hide y-axis ticks 
    # f # to display 
    save("/hm2/code/julia/LearnGLM/figs/ISL_Fig42b.png", f)

end

beta_0 = -10.65
beta_1 = 0.0055

defaultDataFig42b( beta_0, beta_1)


CairoMakie.Screen{IMAGE}


In [8]:
# p(default | income)

function defaultDataFig42i(b0, b1)

    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Default.csv", "NO")

    dfd   = dfConfig( df, [2] )   # get Default
    dfi   = dfConfig( df, [5] )   # get Income

    nr = nrow(dfd)

    X = Matrix( dfi ) 
    Y = Matrix( dfd )
    ylr = zeros(Float64, nr)

     for i = 1:nr 
        num = exp(b0 + b1*X[i])
        ylr[i] = num / (1.0 + num)
        # ylr[i] = 1.0 - ylr[i] 
     end

    f = Figure()
    ax = Makie.Axis(f[1, 1], title="Log Reg of    Default | Income " , xlabel="Income", ylabel="Default")
    scatter!(ax, marker=:circle, markersize=3, color=:blue,   X[1:10000], ylr[1:10000] ) # Default | Income 
    # lines!(ax, X[1:10000], ylr[1:10000])
    
    ax = Makie.Axis(f[1, 1])
    scatter!(ax, marker=:circle, markersize=3, color=:red,   X[1:10000], Y[1:10000] ) # Default | Income 
    hideydecorations!(ax, ticks = false)  # Hide y-axis ticks 
    
    # f # to display 
    save("/hm2/code/julia/LearnGLM/figs/ISL_Fig42i.jpg", f)

end

# for default | income the betas are
beta_0 = -3.09 ; beta_1 = 0.00008


8.0e-5

In [9]:

#=
          function defaultDataTable44()

  Reproduce Table 4.4,, pp. 148 of the ISLR V2 Book.

  The method is presented in section 4.4.2 of the ISLR book, where the LDA classifier is 
  extended to the case of multiple predictors.  X = [ X1, X2, ..., Xp] and it is drawn from 
  a multi-variate Gaussian distribution, with a class-specific mean vector and a common  covariance 
  matrix. Each column in X follows its own one0dimentional normal distribution, but there is some
  "correlation" between each pair of predictors. The function computes LDA on the ISLR V2 Default
  Credit data set, that  contains 10,000 rows with the following
  columns:   
            ColNumber default	student	balance	income
 
  The default column contains "YES" / "NO" values that need to be translated into 0.0 / 1.0 

  Notes From

              https://juliastats.org/MultivariateStats.jl/dev/lda/#MultivariateStats.MulticlassLDA
  
  LDA is expressed as

      w = alfa(Cp + Cn)^-1 (MUp - MUn) =  alpha * (MUp - MUn) / (Cp + Cn)

      f(x) = wTx + b

  The Julia pkg StatsAPI.PI.fit( LinearDiscriminant, Xp, Xn, covestimator=SimpleCovariance()  )                

Following code from 

https://juliaai.github.io/DataScienceTutorials.jl/isl/lab-4/#lda

Fix:

  https://stackoverflow.com/questions/68092206/lda-on-julia-using-multivariatestats

=#


function defaultDataTable44()

    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Default.csv", "NO")
    nr = size(df, 1)
    nc = size(df, 2)
    print("df size =", nr, "  nc =", nc, "\n")
  
    dfc   = dfConfig( df, [1] )   # get column 1, which contains row number
    dfd   = dfConfig( df, [2] )   # get Default
    dfs   = dfConfig( df, [3] )   # get Student
    dfb   = dfConfig( df, [4] )   # get Balance
    dfi   = dfConfig( df, [5] )   # get Income 
  
    ### Replace the elements in Column 1 so that they contain 1.0 as number
    nr = size(dfc,1)
    dfc = DataFrame(ones(nr, 1), :auto)
  
    ### Replace the elements of type string that contain "Yes" or "No" with a number
    ### so that logit regression could be computed on this data.
    
    #replace!(dfd.Default, "Yes" => "1")
    #replace!(dfd.Default, "No"  => "0")
    #dfd[!, :Default] = parse.(Int,dfd[!, :Default])
    
    println("Table 4.4")
      
    # describe does not work anymore 
    # print(df)
    
    xDf = hcat( dfs, dfb, dfi)
  
    X = Matrix(xDf) 
    Y = Matrix( dfd )
  
    #lda = fit(MulticlassLDA, X, Y; outdim=2)
    #Ylda = predict(lda, X)
  
    lda = fit(MulticlassLDA, xDf, dfd, outdim=2)
  
  
  
    ##  Defined globally. The @load macro is defined by MLJ
    ##  BayesianLDA = @load BayesianLDA pkg=MultivariateStats)
    #BayesianLDA = @load BayesianLDA pkg=MultivariateStats
    #classif = machine(BayesianLDA(), X, yc)
    ## as of 2022_1207 a problem exists here
    #fit!(classif)
    print("defaultDataTable44() completed execution")
    
    end
  
    

defaultDataTable44 (generic function with 1 method)

In [10]:

#=
            function doLab4_Part1()
    Reproduce part of Lab4 from the ISLR_V2 book, which is based on Stock Market Data. 
    The section reproduced here is 4.7.1, wherein LogReg is done
    The data consists of percentage returns for the S&P 500 stock
    index over 1, 250 days, from the beginning of 2001 until the end of 2005. For 
    each date, we have recorded the percentage returns for each of the five previous
    trading days, Lag1 through Lag5. We have also recorded Volume (the number of 
    shares traded on the previous day, in billions), Today (the percentage return
    on the date in question) and Direction (whether the market was Up or Down on this
    date). Our goal is to predict Direction (a qualitative response) using the other
    features

  fm = @formula(Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume)
  logit = glm(fm, dfs, Binomial(), ProbitLink())
 
=#
function doLab4_Part1()

    println()
    println("This is DoLab4_Part1")
  
    ## /home/juan/Data/ML_Data/ISL_Data/ISLR2_1.3-2/csv/
    ## Columns = {RECORD	Year	Lag1	Lag2	Lag3	Lag4	Lag5	Volume	Today	Direction}
    ## nRows = 1250
  
    df = readCSVFile("/hm2/Data/ML_Data/ISL_Data/Smarket.csv", "NO")
   
    ### The code below shows how to select specific columns from a DataFrame.
    dfc   = dfConfig( df, [1] )   # get column 1, which contains row number
    dfy   = dfConfig( df, [2] )   # get Year
    df1   = dfConfig( df, [3] )   # get Lag1
    df2   = dfConfig( df, [4] )   # get Lag2
    df3   = dfConfig( df, [5] )   # get Lag3 
    df4   = dfConfig( df, [6] )   # get Lag4 
    df5   = dfConfig( df, [7] )   # get Lag5 
    dfv   = dfConfig( df, [8] )   # get Volume
    dft   = dfConfig( df, [9] )   # get Today
    dfd   = dfConfig( df, [10] )  # get Direction

    ### the code below was used to test code changes for Julia 1.10.1
    #df_test = hcat( df1, df5)
    #println("df_test names ", names( df_test) )
    #println("df_test size ",  size(df_test))
    #println( describe(df_test))
    #fm = @formula(Lag1 ~ Lag5)
    #logit = lm( fm, df_test)
    #println(logit)


    ### Replace the elements in Column 1 so that they contain 1.0 as number
    #nr = size(1, dfc)
    #dfc = DataFrame(ones(nr, 1), :auto)
  
    ### Replace the elements of type string that contain "Up" or "Down" in 
    ### column Direction with a number [0,1] to compute LogReg on the data.
    replace!(dfd.Direction, "Up" => "1")
    replace!(dfd.Direction, "Down" => "0")
    dfd[!, :Direction] = parse.(Int,dfd[!, :Direction])
  
    ### dfs is  Stock Market DataFrame
    ### for the calculations in pp. 173 of the ISLRV2 book, it makes no difference
    ### if dfc and/or dft are added to dfs, so probably it is more efficient to keep
    ### them out of dfs
    dfs = hcat( dfc, dfy, df1, df2, df3, df4, df5, dfv, dft, dfd)
    dfs = hcat( dfy, df1, df2, df3, df4, df5, dfv, dft, dfd) 
    dfs = hcat( dfc, dfy, df1, df2, df3, df4, df5, dfv, dfd)
    dfs = hcat( dfy, df1, df2, df3, df4, df5, dfv, dfd)

    println("Names", names(dfs))
    println("Size=", size(dfs) )
    println( describe(dfs))
    
    ## produce same result as with dfs
    # dfn = hcat(df1, df2, df3, df4, df5, dfv, dfd)

    ### The formula below produces same results as pp. 173 from the ISLR2 book.
    ### Note that ProbitLink() is NOT used in the "R" code of the book
    fm = @formula(Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume)
    logit = glm(fm, dfs, Binomial())
    println(logit)

    # summary( logit )

    model = lm(@formula(Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume), dfs)#  + Lag2 + Lag3 + Lag4 + Lag5 + Volume))


    ### FOR PREDICT https://www.machinelearningplus.com/julia/logistic-regression-in-julia-practical-guide-with-examples/
    ###  https://www.geeksforgeeks.org/logistic-regression-in-julia/

    cf = coef(model)
    #r = round.(coef(model); digits=8)
    r = round.( cf; digits=8)
    println("cf  = ", cf, "\n")
    println("r   = ", r, "\n")

    # test_data = DataFrame(dfd[1000:1250])
    predict = GLM.predict(model, dfs)
    println("Predict=")
    println(predict)

    ### Adding ProbitLink() to the formula produces results that are different from
    ### the results in pp. 173 from the ISLR2 book.  
    # logit = glm(fm, dfs, Binomial(), ProbitLink())

    @show(typeof(dfd))
    @show(names(dfd))
    @show(size(dfd))
    println(dfd[1:5,1])
    # test_data = dfd[1000:1250,1]
    # predict = GLM.predict(logit, test_data)
    # pr = GLM.predict(fm, dfd)

    println("This is the end of DoLab4_Part1")
    println()
  
  end
  
  doLab4_Part1()
  


This is DoLab4_Part1
Names["Year", "Lag1", "Lag2", "Lag3", "Lag4", "Lag5", "Volume", "Direction"]
Size=(1250, 8)
8×7 DataFrame
 Row │ variable   mean          min         median      max         nmissing  eltype   
     │ Symbol     Float64       Real        Float64     Real        Int64     DataType 
─────┼─────────────────────────────────────────────────────────────────────────────────
   1 │ Year       2003.02       2001        2003.0      2005               0  Int64
   2 │ Lag1          0.0038344    -4.922       0.039       5.733           0  Float64
   3 │ Lag2          0.0039192    -4.922       0.039       5.733           0  Float64
   4 │ Lag3          0.001716     -4.922       0.0385      5.733           0  Float64
   5 │ Lag4          0.001636     -4.922       0.0385      5.733           0  Float64
   6 │ Lag5          0.0056096    -4.922       0.0385      5.733           0  Float64
   7 │ Volume        1.4783        0.35607     1.42295     3.15247         0  Float64
   8 │ D

#### GLM Predict



In [11]:
## Try to do prediction following the example from the GLM Documentation 
data = DataFrame(X=[1,2,3], y=[2,4,7]);
mdl = lm(@formula(y ~ X), data);

println("coefs")
cf = round.(coef(mdl); digits=8)
print("coefs=", cf, "\n" )

round(r2(mdl); digits=8)
# round(aic(mdl); digits=8)

test_data = DataFrame(X=[4]);
@show(describe(test_data))
println(test_data)
round.(predict(mdl, test_data); digits=8)


coefs
coefs=[-0.66666667, 2.5]
describe(test_data) = 1×7 DataFrame
 Row │ variable  mean     min    median   max    nmissing  eltype
     │ Symbol    Float64  Int64  Float64  Int64  Int64     DataType
─────┼──────────────────────────────────────────────────────────────
   1 │ X             4.0      4      4.0      4         0  Int64
1×1 DataFrame
 Row │ X     
     │ Int64 
─────┼───────
   1 │     4


1-element Vector{Float64}:
 9.33333333